In [84]:
import os
from time import time
import datatable as dt
from datatable import f, join, by
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datatable import f, by
from sklearn.feature_selection import VarianceThreshold, SelectKBest

In [3]:
base_dir = "./2021_3_data"
test_data_dir  = os.path.join(base_dir, "testdata")
train_data_dir = os.path.join(base_dir, "traindata")

train_dirs = os.listdir(train_data_dir)
test_dirs  = os.listdir(test_data_dir)

action_dirs = os.listdir(os.path.join(train_data_dir, "history_behavior_data"))
action_files = {e: os.listdir(os.path.join(train_data_dir, f"history_behavior_data/{e}"))[0] for e in action_dirs}
print(train_dirs,'\n', test_dirs)

['train.jay', 'all_actions.npz', 'train.npz', 'all_actions.jay', 'user_features_data', 'history_behavior_data', 'video_features_data'] 
 ['test.npz', 'test.jay', 'test.csv', '.ipynb_checkpoints']


In [4]:
user_df = dt.fread(os.path.join(train_data_dir, "user_features_data/user_features_data.csv"), sep='\t')

# random walk
- 通过用户的观看记录的随机游走，生成视频的嵌入
- 通过视频被用户观看的记录的随机游走，生成用户的嵌入

需要考虑以下问题：
1. 随机游走序列的时间跨度是十四天还是时间窗口的形式
2. 随机游走序列能否覆盖所有用户或所有视频
3. 

In [5]:
%%time
all_df = None  # pd.DataFrame([], columns=['user_id', 'video', 'watch_labe', 'is_share'])

for i, d in enumerate(action_dirs):
    df = dt.fread(os.path.join(train_data_dir, f"history_behavior_data/{d}/{action_files[d]}"), sep="\t", )
#                      columns=['user_id', 'video_id', 'is_watch', 'is_sollect', 'is_comment', 'is_share', 'watch_label', 'pt_d'])
    del df[:, 'watch_start_time']
    if all_df is None:
        all_df = df
    else:
        all_df.rbind(df, bynames=True)
print(f"all history data read ...")

all history data read ...
CPU times: user 2min 13s, sys: 12.3 s, total: 2min 25s
Wall time: 28.3 s


In [6]:
all_df.shape

(80276856, 8)

In [30]:
dt.unique(all_df['user_id']).shape, dt.unique(all_df['video_id']).shape

((3953209, 1), (34218, 1))

In [54]:
# 并不是所有的用户都有行为数据，3953209用户有行为数据，每个用户平均有20.03068个视频记录
all_df[:, dt.count(), by("user_id")][:, dt.mean(f.count)]

,count
,▪▪▪▪▪▪▪▪
0,20.3068


In [57]:
# 视频记录次数最少的用户只有 1 次，最多的有 76 次
all_df[:, dt.count(), by("user_id")][:,dt.max(f.count), dt.min(f.count)]

,count,count.0
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,76


In [61]:
# 出现的视频总数 34218，其中还包括没有在视频特征中出现的视频，每个视频平均出现的次数是 2346.04，最多出现了1932565次，最少出现了 963 次
all_df[:, dt.count(), by("video_id")][:, dt.min(f.count), dt.max(f.count)]

,count,count.0
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1932565,963


# 用户上下文
为每个用户每天生成一个上下文向量。
对于用户$u$，其第$t$天的上下文向量$c_{u, t}$由$u$的$t-w, t-1$范围内的交互行为产生，$w$是时间窗口大小。

**不能简单的使用视频的特征作为视频的表征。基础特征里有一些特征不适合进行算术运算，有些特征进行算术运算后就失去了原本的含义**
目前方法：
- 选择一部分列作为视频的表征，基于方差选择
- 选择大部分列作为视频特征再进行降维作为视频表征
- 深度学习

In [129]:
# 每个strategy将一个行为向量转化为一个未归一化的参数
# 每个行为向量是一个向量
def mean_strategy(action):
    assert len(action) > 0
    return sum(action) / len(action)


def weight_strategy(action, weight=None):
    assert len(action) > 0
    if weight is None:
        weight = np.array([1/len(action)] * len(action))
    
    assert len(weight) == len(action)
    act = np.array(action)
    w = act * weight
    
    return sum(w)
    

def cal_context(actions, embeds, strategy, **kwargs):
    """
    根据交互行为，生成上下文向量
    """
    context = None
    w = list(map(lambda x: strategy(x, **kwargs), actions))
    z = sum(w)
    
    embs = np.array(embeds)
    w = np.array(w)
    w = w.reshape((len(w), 1))
    w_embs = np.multiply(embs, w).sum(axis=0)
    w_embs = w_embs / z
    
    return w_embs

In [131]:
actions = [[0, 1, 0, 1], [0, 1, 0, 1]]
embeds = [[1,2,3,4], [1, 2,1, 1]]
kwargs = {'weight': [0.1, 0.2, 0.3, 0.4]}
cal_context(actions, embeds, weight_strategy, **kwargs)

[[0.6]
 [0.6]]


array([1. , 2. , 2. , 2.5])

In [4]:
action_files

{'20210502': 'part-00000-d412c602-2f4a-4649-a81f-e56435dd49fd-c000.csv',
 '20210423': 'part-00000-9809d73a-a55f-4ac2-a59b-9b83cbc5028e-c000.csv',
 '20210424': 'part-00000-225e55dc-4504-4c14-b289-322312355b2b-c000.csv',
 '20210421': 'part-00000-c15f29da-6b1e-48c0-b7d0-2cd560998c3f-c000.csv',
 '20210430': 'part-00000-2da4c3a0-2fcc-422d-8b7c-48940da315ad-c000.csv',
 '20210427': 'part-00000-9132ab46-51c3-4cc3-97de-e7ad5312b852-c000.csv',
 '20210419': 'part-00000-236b99d5-456a-42b2-bd8d-3cbd61d21cc6-c000.csv',
 '20210428': 'part-00000-fc8c8ca1-e655-4a45-b179-c8d9e2dd804c-c000.csv',
 '20210429': 'part-00000-c5dbd994-54d7-4734-adea-0f22d75b23d3-c000.csv',
 '20210422': 'part-00000-3d97d0f8-2572-45e6-bb60-f367c97e7870-c000.csv',
 '20210425': 'part-00000-9d23862b-6bbf-48c6-a598-572df1359737-c000.csv',
 '20210501': 'part-00000-56b6f0ff-57b8-41ac-96ec-c20e3306297e-c000.csv',
 '20210426': 'part-00000-0d315342-3ba7-4727-b4a2-123a1a004786-c000.csv',
 '20210420': 'part-00000-aad75aa4-b60b-4f5b-8def-c4

In [31]:
i = action_dirs[0]
tab1 = dt.fread(os.path.join(train_data_dir, f"history_behavior_data/{i}/{action_files[i]}"), sep="\t")

In [32]:
del tab1[:, 'watch_start_time']

In [33]:
af1 = tab1.to_pandas()

In [28]:
g = af1.groupby('user_id')

In [112]:
%%time
gp = g.get_group(15)
gp

CPU times: user 3.26 ms, sys: 8.04 ms, total: 11.3 ms
Wall time: 132 ms


,user_id,video_id,is_watch,is_share,is_collect,is_comment,watch_label,pt_d
336550,15,32248,False,False,False,False,0,20210502
336551,15,50299,False,False,False,False,0,20210502
336552,15,2259,False,False,False,False,0,20210502
336553,15,8918,False,False,False,False,0,20210502
336554,15,14984,False,False,False,False,0,20210502
336555,15,46900,False,False,False,False,0,20210502
336556,15,39245,False,False,False,False,0,20210502


In [111]:
gp.loc[:, ['video_id', 'is_watch', 'is_share', 'is_collect', 'is_comment', 'watch_label']]

,video_id,is_watch,is_share,is_collect,is_comment,watch_label
336550,32248,False,False,False,False,0
336551,50299,False,False,False,False,0
336552,2259,False,False,False,False,0
336553,8918,False,False,False,False,0
336554,14984,False,False,False,False,0
336555,46900,False,False,False,False,0
336556,39245,False,False,False,False,0


In [56]:
%%time
tab1[f.user_id==15, :]

CPU times: user 536 ms, sys: 136 ms, total: 672 ms
Wall time: 134 ms


,user_id,video_id,is_watch,is_share,is_collect,is_comment,watch_label,pt_d
,▪▪▪▪,▪▪▪▪,▪,▪,▪,▪,▪▪▪▪,▪▪▪▪
0,15,32248,0,0,0,0,0,20210502
1,15,50299,0,0,0,0,0,20210502
2,15,2259,0,0,0,0,0,20210502
3,15,8918,0,0,0,0,0,20210502
4,15,14984,0,0,0,0,0,20210502
5,15,46900,0,0,0,0,0,20210502
6,15,39245,0,0,0,0,0,20210502


In [53]:
dt.unique(tab1[:, f.user_id])

,user_id
,▪▪▪▪
0,2
1,15
2,19
3,38
4,43
5,44
6,45
7,51
8,63


In [55]:
af1['user_id'].unique().shape

(664594,)

In [60]:
video = dt.fread(os.path.join(train_data_dir, "video_features_data/video_features.jay"))

In [87]:
video[:, dt.max(f.video_duration)]

,video_duration
,▪▪▪▪
0,23536


In [88]:
exclude_cols = ['video_id', 'video_name', 'video_release_year', 'video_release_month', 'video_release_day', 'video_duration']
stds = list(video.sd().to_pandas().to_dict(orient='records')[0].items())

In [94]:
stds = list(filter(lambda x: x[0] not in exclude_cols, stds))
varis = list(map(lambda x: (x[0], x[1]**2), stds))

In [104]:
threshold = 0.015
emb_cols = list(filter(lambda x: x[1] > threshold, varis))
emb_cols = [e[0] for e in emb_cols]
emb_cols

['desc_0',
 'desc_3',
 'desc_4',
 'desc_8',
 'desc_9',
 'tags_0',
 'tags_3',
 'tags_4',
 'tags_6',
 'tags_8',
 'class_0',
 'class_2',
 'class_5',
 'class_8',
 'da_0',
 'da_1',
 'da_2',
 'da_3',
 'da_4']